In [2]:
import geopandas as gpd

import shapely
from shapely.geometry import Polygon, MultiPolygon

In [1]:
# create a polygon boundary


In [ ]:
from shapely.geometry import Polygon, MultiPolygon
from typing import Union

def shapely_to_poly_string(
    geom: Union[Polygon, MultiPolygon],
    name: str = "poly_area",
) -> str:
    """
    Convert a shapely Polygon or MultiPolygon to a .poly format string.

    Parameters
    ----------
    geom : shapely.geometry.Polygon or MultiPolygon
        The geometry to convert.
    name : str
        The name of the polygon area (first line in the .poly file).

    Returns
    -------
    str
        The content of the .poly file as a formatted string.
    """
    if isinstance(geom, Polygon):
        polygons = [geom]
    elif isinstance(geom, MultiPolygon):
        polygons = list(geom.geoms)
    else:
        raise TypeError("Input must be a Polygon or MultiPolygon")

    IND1 = " " * 4         # indent for ring label (e.g., 1, !1_1)
    IND2 = " " * (4 * 2)   # indent for coordinates

    lines = [name]

    for i, poly in enumerate(polygons, start=1):
        # Outer ring
        lines.append(f"{IND1}polygon_{i}")
        coords = list(poly.exterior.coords)

        if coords[0] != coords[-1]:
            # ensure ring is closed
            coords.append(coords[0])

        for x, y in coords:
            # 1.1 millimeters
            lines.append(f"{IND2}{x:.8f} {y:.8f}")

        lines.append(f"{IND1}END")

        # Holes (interiors)
        for j, hole in enumerate(poly.interiors, start=1):

            lines.append(f"{IND1}!polygon_{i}_hole_{j}")
            hole_coords = list(hole.coords)

            if hole_coords[0] != hole_coords[-1]:
                hole_coords.append(hole_coords[0])

            for x, y in hole_coords:
                lines.append(f"{IND2}{x:.8f} {y:.8f}")
            lines.append(f"{IND1}END")

    lines.append("END")
    return "\n".join(lines)